In [1]:
!echo $USER | sudo -S systemctl restart nvargus-daemon

[sudo] password for jetson: 

In [2]:
%cd csc-3002-efficient-autonomous-obstacles-avoidance-on-jetracer-1-2025/notebooks/train_collision_avoidance/

/home/jetson/csc-3002-efficient-autonomous-obstacles-avoidance-on-jetracer-1-2025/notebooks/train_collision_avoidance


### Test Collision Avoidance model
This notebook is used to test whether Collision Avoidance model can work.

In [3]:
import cv2
import torch
from torch2trt import TRTModule
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import PIL.Image
import numpy as np
device = torch.device('cuda')

In [4]:
# Load optimized TensorRT model for collision avoidance
model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_model_resnet18_3class_trt.pth'))
model_trt_collision = model_trt_collision.to(device)
model_trt_collision.eval()

TRTModule()

In [5]:
# Define ImageNet mean and std for normalization
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

In [6]:
# Preprocess single input tensor (float16 + normalize)
def preprocess_tensor(image_tensor):
    image_tensor = image_tensor.to(device).half()
    image_tensor.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image_tensor.unsqueeze(0)

In [7]:
# Standard transform for test dataset loading
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
])

In [8]:
# Load test images dataset using ImageFolder structure
test_dataset = ImageFolder('js_data/', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
class_names = test_dataset.classes # Get list of class names from folder names

In [9]:
# Initialize counters
correct = 0
total = len(test_dataset)

print("Running Collision Avoidance Model Evaluation:\n")

# Iterate through test images
for inputs, labels in test_loader:
    inputs = preprocess_tensor(inputs[0]) # preprocess single image
    labels = labels.to(device)

    # Run inference
    with torch.no_grad():
        outputs = model_trt_collision(inputs)
        _, predicted = torch.max(outputs, 1) # get class index with max probability

    pred_class = class_names[predicted.item()]
    true_class = class_names[labels.item()]
    print(f"Predicted: {pred_class.ljust(8)} | Ground Truth: {true_class}")
    
    # Count correct predictions
    if predicted == labels:
        correct += 1

# Calculate and print accuracy
accuracy = correct / total * 100
print(f"\n Accuracy: {correct}/{total} = {accuracy:.2f}%")

Running Collision Avoidance Model Evaluation:

Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_forward | Ground Truth: 0_forward
Predicted: 0_